In [4]:
import MySQLdb
from MySQLdb import Error
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
db_name = os.getenv('DATABASE')
host_name = os.getenv('DATABASE_HOST')
user_name = os.getenv('DATABASE_USERNAME')
user_password = os.getenv('DATABASE_PASSWORD')

In [7]:
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = MySQLdb.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            db=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection


In [9]:
def create_table(connection, create_table_sql):
    cursor = connection.cursor()
    try:
        cursor.execute(create_table_sql)
        print("Table created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [11]:
conn = create_connection(host_name, user_name, user_password, db_name)

Connection to MySQL DB successful


In [27]:

create_user_feedbacks_table = """
CREATE TABLE IF NOT EXISTS user_feedbacks (
    feedback_id CHAR(36) PRIMARY KEY,
    mission_id CHAR(36),
    user_rating INT,
    user_comments TEXT,
    modification_details TEXT,
    timestamp DATETIME,
    prompt_version VARCHAR(255)
);
"""

In [28]:
create_table(conn, create_user_feedbacks_table)


Table created successfully


In [34]:
import faker

In [31]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""Tu es un project owner qui vient de soumettre à l'écrit la mission suivante sur une plateforme de freelance boostée par IA : {mission}"""

In [ ]:
{"role" : "system",
"content" : f"As a project owner that just submitted a mission on the platform,\
you benefit from an ai assistant that helps you to extract technical details from that mission so you can frame your needs rapidly and accurately.\
This time, the AI was not performant enough and you decide to fill a survey to give a feedback. Here is your original posted mission : {mission} ",
"role" : "user",
"content" : processed_mission}

In [36]:
function = {"name" : "feedback",
"description" : "A function that takes in a mission processed by the AI and returns a list of feedback given by the user",
"parameters" : {
    "type" : "object",
    "properties" : {
        "user_rating" : {
            "type" : "integer",
            "description" : "The rating given by the user",
            "minimum" : 1,
            "maximum" : 5
        },
        "user_comments" : {
            "type" : "string",
            "description" : "The comments given by the user",
            "minLength" : 30,
            "maxLength" : 500
        },
        "modification_details" : {
            "type" : "string",
            "description" : "The mission details given by AI modified/rectified by user, can be fully modified or partially",
        },
        "prompt_version" : {
            "type" : "string",
    }

    }
}